In [1]:
# importing libraries
import cv2
import numpy as np
from skimage import io as skimage_io
from skimage.measure import shannon_entropy
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mutual_info_score
import matplotlib.pyplot as plt

In [2]:
def calculate_entropy(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return shannon_entropy(gray_image)

In [3]:
def calculate_clarity(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sobel_x = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
    sobel_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    clarity = np.mean(sobel_magnitude)
    return clarity

In [4]:
def calculate_registration_error(original_points, transformed_points):
    if len(original_points) != len(transformed_points):
        raise ValueError("Points lists must have the same length.")
    errors = np.sqrt(np.sum((np.array(original_points) - np.array(transformed_points))**2, axis=1))
    return np.mean(errors)

In [5]:
def calculate_mutual_information(reference_image, mosaic_image):
    hist_2d, _, _ = np.histogram2d(reference_image.ravel(), mosaic_image.ravel(), bins=20)
    mi = mutual_info_score(None, None, contingency=hist_2d)
    return mi

In [6]:
def calculate_psnr(reference_image, mosaic_image, max_psnr=100):
    mse = np.mean((reference_image - mosaic_image) ** 2)
    if mse == 0:
        return max_psnr
    PIXEL_MAX = 255.0
    return 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

In [12]:
def calculate_ssim(reference_image, mosaic_image):
    try:
        # Convert images to grayscale
        reference_image_gray = cv2.cvtColor(reference_image, cv2.COLOR_BGR2GRAY)
        mosaic_image_gray = cv2.cvtColor(mosaic_image, cv2.COLOR_BGR2GRAY)
        
        # Ensure window size is appropriate
        min_dim = min(reference_image_gray.shape[0], reference_image_gray.shape[1],
                      mosaic_image_gray.shape[0], mosaic_image_gray.shape[1])
        win_size = min(7, min_dim - 1 if min_dim % 2 == 0 else min_dim)
        
        return ssim(reference_image_gray, mosaic_image_gray, win_size=win_size)
    except Exception as e:
        # Print the error message and return the worst SSIM value
        print(f"Error calculating SSIM: {e}")
        return 0

In [8]:
def resize_image(image, target_shape):
    return cv2.resize(image, (target_shape[1], target_shape[0]), interpolation=cv2.INTER_AREA)

def evaluate_mosaic(reference_image_path, mosaic_image_path, reference_points, transformed_points):
    # Load images
    reference_image = skimage_io.imread(reference_image_path)
    mosaic_image = skimage_io.imread(mosaic_image_path)
    
    # Resize reference image to match mosaic image dimensions
    reference_image_resized = resize_image(reference_image, mosaic_image.shape)
    
    # Calculate metrics
    entropy = calculate_entropy(mosaic_image)
    clarity = calculate_clarity(mosaic_image)
    # registration_error = calculate_registration_error(reference_points, transformed_points)
    psnr_value = calculate_psnr(reference_image_resized, mosaic_image)
    ssim_value = calculate_ssim(reference_image_resized, mosaic_image)
    mutual_information = calculate_mutual_information(reference_image_resized, mosaic_image)
    
    return (entropy, clarity, psnr_value, ssim_value, mutual_information)